'C:\\Users\\chadm\\Desktop\\tides_calendar_code'

In [2]:
import numpy as np
import pandas as pd
import os
import datetime
import xml.etree.ElementTree as ET
import holidays

In [21]:
tides_year = '2021'
annual_tides = ET.parse(f'mry_tides_annual{tides_year}.xml').getroot()

In [22]:
col_names = list()
for n in range(0,6):
    col_names.append(annual_tides[-1][0][n].tag)
col_names.remove('date')
col_names.remove('time')
col_names.insert(0,'date_time')
col_names

['date_time', 'day', 'pred_in_ft', 'pred_in_cm', 'highlow']

In [23]:
f = annual_tides.find(annual_tides[-1].tag)
df_mry_tides21 = pd.DataFrame(index=range(0,len(list(f.iter(annual_tides[-1][0].tag)))),columns=col_names)
df_mry_tides21

,date_time,day,pred_in_ft,pred_in_cm,highlow
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
1406,NaN,NaN,NaN,NaN,NaN
1407,NaN,NaN,NaN,NaN,NaN
1408,NaN,NaN,NaN,NaN,NaN
1409,NaN,NaN,NaN,NaN,NaN


In [24]:
for i in df_mry_tides21.index:
    df_mry_tides21['date_time'].iloc[i]=datetime.datetime.strptime(annual_tides[-1][i][0].text+annual_tides[-1][i][2].text,'%Y/%m/%d%I:%M %p')
    df_mry_tides21[annual_tides[-1][0][1].tag].iloc[i]=annual_tides[-1][i][1].text
    df_mry_tides21[annual_tides[-1][0][3].tag].iloc[i]=float(annual_tides[-1][i][3].text)
    df_mry_tides21[annual_tides[-1][0][4].tag].iloc[i]=float(annual_tides[-1][i][4].text)
    df_mry_tides21[annual_tides[-1][0][5].tag].iloc[i]=annual_tides[-1][i][5].text

df_mry_tides21

,date_time,day,pred_in_ft,pred_in_cm,highlow
0,2022-01-01 01:50:00,Sat,2.7,82.0,L
1,2022-01-01 08:13:00,Sat,6.89,210.0,H
2,2022-01-01 15:49:00,Sat,-1.64,-50.0,L
3,2022-01-01 22:41:00,Sat,3.96,121.0,H
4,2022-01-02 02:45:00,Sun,2.78,85.0,L
...,...,...,...,...,...
1406,2022-12-30 22:38:00,Fri,1.23,37.0,L
1407,2022-12-31 05:29:00,Sat,5.4,165.0,H
1408,2022-12-31 12:37:00,Sat,0.95,29.0,L
1409,2022-12-31 18:30:00,Sat,3.23,98.0,H


In [25]:
df_copy = df_mry_tides21.copy()
for i in df_copy.index:
    if not df_copy['highlow'][i] == 'L':
        df_copy.drop(index=i,inplace=True)
    else:
        if df_copy['day'][i] == 'Sat' or df_copy['day'][i] =='Sun' or df_copy['date_time'][i].date() in holidays.US(years=df_copy['date_time'][i].date().year):
            if not 8 <= df_copy['date_time'][i].hour <= 20:
                df_copy.drop(index=i,inplace=True)
            else: pass
        else:
            if not 16 <= df_copy['date_time'][i].hour <= 20:
                df_copy.drop(index=i,inplace=True)
            else: pass
for i in df_copy.index:
    if df_copy['pred_in_cm'][i]>=df_copy['pred_in_cm'].mean():
        df_copy.drop(index=i,inplace=True)
    else: pass
df_copy

,date_time,day,pred_in_ft,pred_in_cm,highlow
2,2022-01-01 15:49:00,Sat,-1.64,-50.0,L
6,2022-01-02 16:37:00,Sun,-1.84,-56.0,L
10,2022-01-03 17:25:00,Mon,-1.81,-55.0,L
14,2022-01-04 18:12:00,Tue,-1.56,-48.0,L
18,2022-01-05 18:58:00,Wed,-1.14,-35.0,L
...,...,...,...,...,...
1378,2022-12-23 16:52:00,Fri,-1.69,-52.0,L
1382,2022-12-24 17:40:00,Sat,-1.74,-53.0,L
1386,2022-12-25 18:29:00,Sun,-1.58,-48.0,L
1390,2022-12-26 19:18:00,Mon,-1.23,-37.0,L


In [26]:
df_goog_cal = pd.DataFrame(columns=['Subject','Start Date','Start Time','End Date','End Time','All Day Event','Private','Description','Location'],index=df_copy.index)
for i in df_goog_cal.index:
    df_goog_cal['Subject'].loc[i] = 'Low Tide'
    df_goog_cal['Start Date'].loc[i] = df_copy['date_time'][i].date()
    df_goog_cal['Start Time'].loc[i] = df_copy['date_time'][i].time()
    df_goog_cal['End Date'].loc[i] = df_copy['date_time'][i].date()
    df_goog_cal['End Time'].loc[i] = df_copy['date_time'][i].time()
    df_goog_cal['All Day Event'].loc[i] = False
    df_goog_cal['Private'].loc[i] = False
    df_goog_cal['Description'].loc[i] = 'Below Average Low Tide'
    df_goog_cal['Location'].loc[i] = 'Monterey, CA'
df_goog_cal

,Subject,Start Date,Start Time,End Date,End Time,All Day Event,Private,Description,Location
2,Low Tide,2022-01-01,15:49:00,2022-01-01,15:49:00,False,False,Below Average Low Tide,"Monterey, CA"
6,Low Tide,2022-01-02,16:37:00,2022-01-02,16:37:00,False,False,Below Average Low Tide,"Monterey, CA"
10,Low Tide,2022-01-03,17:25:00,2022-01-03,17:25:00,False,False,Below Average Low Tide,"Monterey, CA"
14,Low Tide,2022-01-04,18:12:00,2022-01-04,18:12:00,False,False,Below Average Low Tide,"Monterey, CA"
18,Low Tide,2022-01-05,18:58:00,2022-01-05,18:58:00,False,False,Below Average Low Tide,"Monterey, CA"
...,...,...,...,...,...,...,...,...,...
1378,Low Tide,2022-12-23,16:52:00,2022-12-23,16:52:00,False,False,Below Average Low Tide,"Monterey, CA"
1382,Low Tide,2022-12-24,17:40:00,2022-12-24,17:40:00,False,False,Below Average Low Tide,"Monterey, CA"
1386,Low Tide,2022-12-25,18:29:00,2022-12-25,18:29:00,False,False,Below Average Low Tide,"Monterey, CA"
1390,Low Tide,2022-12-26,19:18:00,2022-12-26,19:18:00,False,False,Below Average Low Tide,"Monterey, CA"


In [27]:
df_goog_cal.to_csv(f'google_cal_tides{tides_year}.csv')